In [ ]:
# download nltk stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Install a particular version of `google-cloud-storage` because (oddly enough) 
# the  version on Colab and GCP is old. A dependency error below is okay.
!pip install -q google-cloud-storage==1.43.0

     |████████████████████████████████| 106 kB 9.7 MB/s 
     |████████████████████████████████| 75 kB 4.0 MB/s 
     |████████████████████████████████| 109 kB 39.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-translate 1.5.0 requires google-api-core[grpc]<2.0.0dev,>=1.6.0, but you have google-api-core 2.3.2 which is incompatible.
google-cloud-translate 1.5.0 requires google-cloud-core<2.0dev,>=1.0.0, but you have google-cloud-core 2.2.1 which is incompatible.
google-cloud-language 1.2.0 requires google-api-core[grpc]<2.0.0dev,>=1.6.0, but you have google-api-core 2.3.2 which is incompatible.
google-cloud-firestore 1.7.0 requires google-api-core[grpc]<2.0.0dev,>=1.14.0, but you have google-api-core 2.3.2 which is incompatible.
google-cloud-firestore 1.7.0 requires google-cloud-core<2.0dev,>=1.0.3, but you have google-cloud-core 2.2.1 which

In [ ]:
# authenticate below for Google Storage access as needed
from google.colab import auth
auth.authenticate_user()

In [ ]:
# install ngrok to emulate public IP / address
!wget -N https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip

for details.

--2022-01-04 08:45:54--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.161.241.46, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  19.2MB/s    in 0.7s    

2022-01-04 08:45:55 (19.2 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
# TODO: sign up for an ngrok account
# then put your ngrok token below, uncomment, and execute
!./ngrok authtoken 22KeqcMQYpd2iRnsaZZOzlyaNw0_5NAk69pXRUMvfrf1oCNpU

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# install a ngrok python package and a version of flask that works with it in 
# colab
!pip -q install flask-ngrok
!pip -q install flask==0.12.2
# !pip -q install flask_restful

     |████████████████████████████████| 83 kB 916 kB/s 


# Run the app

In [ ]:
# you need to upload your implementation of search_app.py
import search_frontend as se

In [ ]:
# uncomment the code below and execute to reload the module when you make 
# changes to search_frontend.py (after you upload again).
# import importlib
# importlib.reload(se)

In [ ]:
from flask_ngrok import run_with_ngrok
run_with_ngrok(se.app) 
se.app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c271-34-125-136-123.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [04/Jan/2022 08:48:13] "GET /search?query=hello HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2022 08:48:13] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/Jan/2022 08:49:10] "GET /search_body?query=hello HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2022 08:49:10] "GET /favicon.ico HTTP/1.1" 404 -


# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [13]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [14]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

url = 'http://34.122.57.217:8080'

In [15]:
import requests
from time import time

qs_res = []
av_total = 0
duration_total = 0
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search', {'query': q}, timeout=35)
    duration = time() - t_start
    duration_total += duration
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
      av_total += ap
  except:
    pass
  
  qs_res.append((q, duration, ap))
print("Average av = " + str(av_total / len(qs_res)))
print("Average duration = " + str(duration_total / len(qs_res)))
qs_res

Average av = 0.4740333333333334
Average duration = 1.7774594306945801


[('best marvel movie', 2.940922737121582, 0.388),
 ('How do kids come to world?', 4.976233959197998, 0.189),
 ('Information retrieval', 1.1563773155212402, 0.861),
 ('LinkedIn', 0.014243364334106445, 0.862),
 ('How to make coffee?', 1.7711775302886963, 0.447),
 ('Ritalin', 0.006956338882446289, 0.796),
 ('How to make wine at home?', 4.241507530212402, 0.454),
 ('Most expensive city in the world', 7.360634803771973, 0.193),
 ('India', 0.9725799560546875, 0.129),
 ('how to make money fast?', 2.6443889141082764, 0.568),
 ('Netflix', 0.06100630760192871, 0.561),
 ('Apple computer', 0.511143684387207, 0.469),
 ('The Simpsons', 0.027717113494873047, 0.23),
 ('World cup', 5.112840414047241, 0.7),
 ('How to lose weight?', 0.5067093372344971, 0.449),
 ('Java', 0.07606220245361328, 0.3),
 ('Air Jordan', 1.3025891780853271, 0.731),
 ('how to deal with depression?', 0.7114245891571045, 0.31),
 ('How do you make gold', 2.5193991661071777, 0.259),
 ('Marijuana', 0.03547978401184082, 0.29),
 ('How to

In [6]:
import requests
from time import time

qs_res = []
av_total = 0
duration_total = 0
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search_body', {'query': q}, timeout=35)
    duration = time() - t_start
    duration_total += duration
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
      av_total += ap
  except:
    pass
  
  qs_res.append((q, duration, ap))
print("Average av = " + str(av_total / len(qs_res)))
print("Average duration = " + str(duration_total / len(qs_res)))
qs_res

Average av = 0.3849666666666666
Average duration = 1.4953412373860677


[('best marvel movie', 2.449080467224121, 0.221),
 ('How do kids come to world?', 4.345254898071289, 0.037),
 ('Information retrieval', 0.9845311641693115, 0.376),
 ('LinkedIn', 0.01553654670715332, 0.815),
 ('How to make coffee?', 1.5099046230316162, 0.505),
 ('Ritalin', 0.006654262542724609, 0.712),
 ('How to make wine at home?', 3.372323513031006, 0.249),
 ('Most expensive city in the world', 6.388381004333496, 1.0),
 ('India', 0.9113571643829346, 0.263),
 ('how to make money fast?', 2.1678237915039062, 0.276),
 ('Netflix', 0.05511164665222168, 0.712),
 ('Apple computer', 0.5216565132141113, 0.508),
 ('The Simpsons', 0.02642536163330078, 0.192),
 ('World cup', 4.291909217834473, 0.54),
 ('How to lose weight?', 0.4066128730773926, 0.288),
 ('Java', 0.07217884063720703, 0.407),
 ('Air Jordan', 1.0284132957458496, 0.421),
 ('how to deal with depression?', 0.5825822353363037, 0.323),
 ('How do you make gold', 2.0956597328186035, 0.385),
 ('Marijuana', 0.03460264205932617, 0.081),
 ('How

In [7]:
import requests
from time import time

qs_res = []
av_total = 0
duration_total = 0
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search_title', {'query': q}, timeout=35)
    duration = time() - t_start
    duration_total += duration
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
      av_total += ap
  except:
    pass
  
  qs_res.append((q, duration, _[0]))
print("Average av = " + str(av_total / len(qs_res)))
print("Average duration = " + str(duration_total / len(qs_res)))
qs_res

Average av = 0.43983333333333335
Average duration = 0.0564259926478068


[('best marvel movie',
  0.06739163398742676,
  "Critics' Choice Movie Award for Best Picture"),
 ('How do kids come to world?', 0.19700312614440918, 'Kids World'),
 ('Information retrieval', 0.01874256134033203, 'Information retrieval'),
 ('LinkedIn', 0.005491971969604492, 'LinkedIn'),
 ('How to make coffee?', 0.014283180236816406, 'Make Yourself'),
 ('Ritalin', 0.0054416656494140625, 'Ritalin class-action lawsuits'),
 ('How to make wine at home?',
  0.03967165946960449,
  'The Make Yourself at Home EP'),
 ('Most expensive city in the world',
  0.27289557456970215,
  'List of most expensive streets by city'),
 ('India', 0.036550045013427734, 'Bassas da India'),
 ('how to make money fast?', 0.02160787582397461, 'Make Money Fast'),
 ('Netflix', 0.0059816837310791016, 'Netflix'),
 ('Apple computer',
  0.020171403884887695,
  'Apple Computer, Inc. v. Microsoft Corp.'),
 ('The Simpsons', 0.006436824798583984, 'The Simpsons'),
 ('World cup', 0.3042612075805664, 'Cricket World Cup'),
 ('How 

In [9]:
import requests
from time import time

qs_res = []
av_total = 0
duration_total = 0
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search_anchor', {'query': q}, timeout=35)
    duration = time() - t_start
    duration_total += duration
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
      av_total += ap
  except:
    pass
  
  qs_res.append((q, duration, _[0]))
print("Average av = " + str(av_total / len(qs_res)))
print("Average duration = " + str(duration_total / len(qs_res)))
qs_res

Average av = 0.16460000000000002
Average duration = 0.3720516045888265


[('best marvel movie', 0.6780505180358887, 'Marvel Comics'),
 ('How do kids come to world?', 1.15999436378479, 'Academy Awards'),
 ('Information retrieval', 0.08580732345581055, 'Espionage'),
 ('LinkedIn', 0.021677255630493164, 'Academy Awards'),
 ('How to make coffee?', 0.09036755561828613, 'Academy Awards'),
 ('Ritalin', 0.005834102630615234, 'Methylphenidate'),
 ('How to make wine at home?', 0.29260993003845215, 'Alabama'),
 ('Most expensive city in the world', 2.163865089416504, 'Materialism'),
 ('India', 0.23818111419677734, 'Autism'),
 ('how to make money fast?', 0.08882474899291992, 'Azores'),
 ('Netflix', 0.04771780967712402, 'Achilles'),
 ('Apple computer', 0.2912139892578125, 'Alabama'),
 ('The Simpsons', 0.02617955207824707, 'Animal (disambiguation)'),
 ('World cup', 1.8789489269256592, 'Anarchism'),
 ('How to lose weight?', 0.010123014450073242, 'Dieting'),
 ('Java', 0.045804500579833984, 'Anarchism'),
 ('Air Jordan', 0.5630455017089844, 'Anarchism'),
 ('how to deal with de

In [10]:
import requests
from time import time
lst = [12, 40, 3080, 4500]

pred_wids = []
qs_res = []
duration, ap = None, None
t_start = time()
try:
  res = requests.post(url +'/get_pagerank', json=lst)
  duration = time() - t_start
  if res.status_code == 200:
    pred_wids = res.json()
except:
  pass

print(pred_wids)

[147.7334534423781, 0, 235.82512239484552, 0]


In [11]:
import requests
from time import time
lst = [3390, 737, 307, 308, 330, 316, 324, 25, 340]
pred_wids = []

qs_res = []
duration, ap = None, None
t_start = time()
try:
  res = requests.post(url +'/get_pageview', json=lst)
  duration = time() - t_start
  if res.status_code == 200:
    pred_wids = res.json()
except:
  pass

print(pred_wids)

[4525604, 6039676, 343343, 151041, 39, 7167, 223758, 114746, 1222]
